In [ ]:
#STEP 1- Scraping WikiPedia and Creating Data Frame

# install and import libraries
#neighbourhood maps from open.toronto.ca


# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install folium
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install html5lib 

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import folium
print('DONE!')

# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:

url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "4def3f65-2a65-4a4f-83c4-b2a4aed72d46"}
package = requests.get(url, params = params).json()
#print(package["result"])

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url)
postal_code_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source.text, 'html')
soup1 = BeautifulSoup(postal_code_url,'lxml')
#print(soup1.tbody.prettify())
#print("VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV")
#print(soup.tbody.prettify())

#using soup object, iterate the wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find("tbody")

My_table = soup.find('tbody')
df = pd.read_html(str(My_table))[0]
df
#print(df)

#function for finding italics 
def has_italics_not_assigned(tag):
    return tag.has_attr('i') and not tag.has_attr('a')

for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        if td.find_all(has_italics_not_assigned(td)):
            #print(td)
            break
        else:
            section.append(td.text.strip())
        continue
        #section.append(td.text.strip())
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
Toronto_df = pd.DataFrame(data = data, columns = columns)

#Toronto_df.shape
#Toronto_df = Toronto_df[Toronto_df['Borough'] != 'Not assigned']
#Toronto_df.reset_index(inplace=True)
#Toronto_df.drop('index', axis=1, inplace=True)
#print('This table has {} rows and {} columns.'.format(Toronto_df.shape[0], Toronto_df.shape[1]))

#Toronto_df

In [ ]:
# hi this is an edit hello# what about now? Interesting...